### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [280]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [281]:
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

In [282]:
# set variable for general stats
num_schools = len(school_data_complete["school_name"].unique())
num_students = school_data_complete["Student ID"].count()
total_budget = school_data["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()

In [283]:
# Create bin for pass/fail. Then calc pass/fail rates for each math, reading, and both
pass_bin = [0,69,100]
pass_name = ["Fail","Pass"]
# Math, reading, and both pass rate
school_data_complete["Math_PF"] = pd.cut(school_data_complete["math_score"],pass_bin,labels=pass_name)
count_passing_math = school_data_complete.loc[school_data_complete["Math_PF"] == "Pass"]["Student ID"].count()
pct_passing_math = count_passing_math / num_students * 100

school_data_complete["Read_PF"] = pd.cut(school_data_complete["reading_score"],pass_bin,labels=pass_name)
count_passing_read = school_data_complete.loc[school_data_complete["Read_PF"] == "Pass"]["Student ID"].count()
pct_passing_read = count_passing_read / num_students * 100

count_passing_both = school_data_complete.loc[(school_data_complete["Math_PF"] == "Pass") & (school_data_complete["Read_PF"] == "Pass")]["Student ID"].count()
count_passing_both
pct_passing_both = count_passing_both / num_students *100

In [284]:
# District Summary
district_summary_data = [{"Total Schools": num_schools, "Total Students" : num_students, "Total Budget" : total_budget, "Average Math Score": avg_math_score , "Average Reading Score" : avg_read_score, "% Passing Math": pct_passing_math, "% Passing Reading" : pct_passing_read, "% Overall Passing" : pct_passing_both }]
district_summary_df = pd.DataFrame(district_summary_data)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].astype(float).map("${:,.2f}".format)
district_summary_df['Total Students'] = district_summary_df['Total Students'].astype(float).map("{:,.0f}".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [285]:
# make listing of schools and di
name_of_school = school_data_complete.groupby(['school_name'])['school_name'].first()
type_of_school = school_data_complete.groupby(['school_name'])['type'].first()

# stats per school
school_total_students = school_data_complete.groupby(['school_name'])['Student ID'].count()
school_total_budget = school_data_complete.groupby(['school_name'])['budget'].mean()
school_avg_math = school_data_complete.groupby(['school_name'])['math_score'].mean()
school_avg_read = school_data_complete.groupby(['school_name'])['reading_score'].mean()

school_mathP_count = school_data_complete[school_data_complete['Math_PF'] == "Pass"].groupby(['school_name']).size()
school_readP_count = school_data_complete[school_data_complete['Read_PF'] == "Pass"].groupby(['school_name']).size()
school_bothP_count = school_data_complete[(school_data_complete['Math_PF'] == "Pass") & (school_data_complete['Read_PF'] == "Pass")].groupby(['school_name']).size()


In [296]:
# make school summary data
school_summary_data_df = pd.DataFrame({"School Type": type_of_school, "Total Students": school_total_students,  "Total School Budget": school_total_budget, "Average Math Score": school_avg_math, "Average Reading Score": school_avg_read})

# calc additional stat fields
per_student_budget = school_total_budget / school_total_students
pct_passing_math = school_mathP_count / school_total_students * 100
pct_passing_read = school_readP_count / school_total_students * 100
pct_passing_both = school_bothP_count / school_total_students * 100

# make school summary output table
school_summary_df = pd.DataFrame({"School Type": type_of_school, "Total Students": school_total_students,  "Total School Budget": school_total_budget, "Per Student Budget": per_student_budget, "Average Math Score": school_avg_math, "Average Reading Score": school_avg_read, "% Passing Math": pct_passing_math, "% Passing Reading": pct_passing_read, "% Overall Passing": pct_passing_both})

# format fields
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].astype(float).map("${:,.2f}".format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].astype(float).map("${:,.2f}".format)
school_summary_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [287]:
school_top_5 = school_summary_df.sort_values("% Overall Passing", ascending = False)
school_top_5.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [288]:
school_top_5 = school_summary_df.sort_values("% Overall Passing")
school_top_5.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [289]:
# make listing of schools
avg_math_9 = school_data_complete[school_data_complete['grade'] == "9th"].groupby(['school_name'])['math_score'].mean()
avg_math_10 = school_data_complete[school_data_complete['grade'] == "10th"].groupby(['school_name'])['math_score'].mean()
avg_math_11 = school_data_complete[school_data_complete['grade'] == "11th"].groupby(['school_name'])['math_score'].mean()
avg_math_12 = school_data_complete[school_data_complete['grade'] == "12th"].groupby(['school_name'])['math_score'].mean()

math_summary_by_grade_df = pd.DataFrame({"9th" : avg_math_9, "10th": avg_math_10, "11th": avg_math_11,"12th": avg_math_12})
math_summary_by_grade_df['9th'] = math_summary_by_grade_df['9th'].round(1)
math_summary_by_grade_df['10th'] = math_summary_by_grade_df['10th'].round(1)
math_summary_by_grade_df['11th'] = math_summary_by_grade_df['11th'].round(1)
math_summary_by_grade_df['12th'] = math_summary_by_grade_df['12th'].round(1)
math_summary_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [290]:
# make listing of schools
avg_read_9 = school_data_complete[school_data_complete['grade'] == "9th"].groupby(['school_name'])['reading_score'].mean()
avg_read_10 = school_data_complete[school_data_complete['grade'] == "10th"].groupby(['school_name'])['reading_score'].mean()
avg_read_11 = school_data_complete[school_data_complete['grade'] == "11th"].groupby(['school_name'])['reading_score'].mean()
avg_read_12 = school_data_complete[school_data_complete['grade'] == "12th"].groupby(['school_name'])['reading_score'].mean()

read_summary_by_grade_df = pd.DataFrame({"9th" : avg_read_9, "10th": avg_read_10, "11th": avg_read_11,"12th": avg_read_12})
read_summary_by_grade_df['9th'] = read_summary_by_grade_df['9th'].round(1)
read_summary_by_grade_df['10th'] = read_summary_by_grade_df['10th'].round(1)
read_summary_by_grade_df['11th'] = read_summary_by_grade_df['11th'].round(1)
read_summary_by_grade_df['12th'] = read_summary_by_grade_df['12th'].round(1)
read_summary_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [324]:
# school_summary_df.apply(lambda col:pd.to_numeric(col, errors = 'coerce'))
# school_summary_df.dtypes

# Create bin for spending category. 
spending_bin = [-100,585,630,645,100000]
spending_name = ["<$585","$585-630","$630-645","$645-680"]
school_summary_df["Spending Category"] = pd.cut(school_summary_df["Per Student Budget"],spending_bin,labels=spending_name)




TypeError: '<' not supported between instances of 'int' and 'str'

In [18]:
###  example

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [88]:
list_test = ["1", "2", "3", "4", "a", "b", "c"]
list_test[:5]

['1', '2', '3', '4', 'a']